In [1]:
11

11

## 정보
- data from 2018-01-01 to 2020-12-31

## packages

In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/mydrive/')

Mounted at /content/mydrive/


In [4]:
train = pd.read_csv('/content/mydrive/MyDrive/seoul-bike-rental-prediction/given-datasets/train.csv')
test = pd.read_csv('/content/mydrive/MyDrive/seoul-bike-rental-prediction/given-datasets/test.csv')

## 결측치 처리

In [5]:
# precipitation, PM10, Pm2.5, sunshine_sum에 결측값이 있다.
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1095 non-null   object 
 1   precipitation  417 non-null    float64
 2   temp_mean      1095 non-null   float64
 3   temp_highest   1095 non-null   float64
 4   temp_lowest    1095 non-null   float64
 5   PM10           1028 non-null   float64
 6   PM2.5          1027 non-null   float64
 7   humidity       1095 non-null   float64
 8   sunshine_sum   1090 non-null   float64
 9   sunshine_rate  1095 non-null   float64
 10  wind_mean      1095 non-null   float64
 11  wind_max       1095 non-null   float64
 12  rental         1095 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 111.3+ KB


In [6]:
# PM10, Pm2.5에 결측값이 없다.
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           365 non-null    object 
 1   precipitation  160 non-null    float64
 2   temp_mean      365 non-null    float64
 3   temp_highest   365 non-null    float64
 4   temp_lowest    365 non-null    float64
 5   PM10           365 non-null    float64
 6   PM2.5          365 non-null    float64
 7   humidity       365 non-null    float64
 8   sunshine_sum   364 non-null    float64
 9   sunshine_rate  365 non-null    float64
 10  wind_mean      365 non-null    float64
 11  wind_max       365 non-null    float64
dtypes: float64(11), object(1)
memory usage: 34.3+ KB


### pm10, pm2.5

In [7]:
# PM10, PM2.5에 결측값이 있는 샘플을 train에서 삭제한다. (train에는 있지만 test에는 없기 때문)
train.dropna(axis=0, subset=['PM10', 'PM2.5'], inplace=True)
train.isnull().sum()[train.isnull().sum() > 0]

precipitation    639
sunshine_sum       5
dtype: int64

### sunshine_sum

In [8]:
# sunshine_sum 결측값을 갖는 다섯개 샘플 : 모두 sunshine_rate가 0이다.
train[train['sunshine_sum'].isnull()]

,date,precipitation,temp_mean,temp_highest,temp_lowest,PM10,PM2.5,humidity,sunshine_sum,sunshine_rate,wind_mean,wind_max,rental
17,2018-01-18,0.0,2.1,4.5,0.3,80.0,56.0,59.6,NaN,0.0,1.9,3.8,7054
25,2018-01-26,NaN,-14.8,-10.7,-17.8,28.0,18.0,34.9,NaN,0.0,2.6,4.6,3116
45,2018-02-15,NaN,-0.7,4.0,-3.7,32.0,14.0,28.0,NaN,0.0,2.5,4.4,3935
187,2018-07-07,NaN,23.7,29.2,19.5,7.0,5.0,51.4,NaN,0.0,1.6,2.3,48015
604,2019-08-28,NaN,26.1,30.2,23.6,29.0,21.0,66.2,NaN,0.0,1.9,4.3,88718


In [9]:
# sunshine_rate가 0인 모든 샘플 : 습도가 높고 미세먼지가 많은 날로 추정된다.
train[train['sunshine_rate'] == 0]

,date,precipitation,temp_mean,temp_highest,temp_lowest,PM10,PM2.5,humidity,sunshine_sum,sunshine_rate,wind_mean,wind_max,rental
17,2018-01-18,0.0,2.1,4.5,0.3,80.0,56.0,59.6,NaN,0.0,1.9,3.8,7054
25,2018-01-26,NaN,-14.8,-10.7,-17.8,28.0,18.0,34.9,NaN,0.0,2.6,4.6,3116
45,2018-02-15,NaN,-0.7,4.0,-3.7,32.0,14.0,28.0,NaN,0.0,2.5,4.4,3935
57,2018-02-27,NaN,4.2,6.9,0.6,82.0,55.0,48.8,0.0,0.0,0.9,1.8,10064
58,2018-02-28,25.0,4.6,7.8,1.8,67.0,43.0,73.3,0.0,0.0,2.0,3.7,4230
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,2020-11-01,13.6,13.7,15.0,11.6,41.0,23.0,84.3,0.0,0.0,2.3,4.5,32925
1051,2020-11-18,12.2,16.6,18.7,14.8,45.0,29.0,91.4,0.0,0.0,2.6,3.6,44452
1052,2020-11-19,86.9,13.6,19.1,6.9,19.0,11.0,90.5,0.0,0.0,4.2,7.3,34590
1053,2020-11-20,0.0,2.8,7.0,0.3,23.0,14.0,74.3,0.0,0.0,3.2,5.6,60332


In [10]:
# sunshine_sum의 count  : 90 (cf. NaN은 count에 미포함)
# sunshine_rate의 count : 95
train[train['sunshine_rate'] == 0].describe()

,precipitation,temp_mean,temp_highest,temp_lowest,PM10,PM2.5,humidity,sunshine_sum,sunshine_rate,wind_mean,wind_max,rental
count,87.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,90.0,95.0,95.000000,95.000000,95.000000
mean,23.898851,15.332632,17.961053,13.065263,26.484211,16.989474,79.837895,0.0,0.0,2.194737,4.587368,26079.336842
std,27.208675,8.832742,8.552955,9.427470,17.661030,12.434933,14.647716,0.0,0.0,0.893400,1.570692,21939.621141
min,0.000000,-14.800000,-10.700000,-17.800000,5.000000,4.000000,28.000000,0.0,0.0,0.700000,1.600000,1037.000000
25%,3.150000,7.850000,10.450000,5.500000,13.000000,7.000000,72.550000,0.0,0.0,1.550000,3.450000,9872.000000
50%,12.000000,17.800000,20.200000,15.700000,24.000000,14.000000,83.600000,0.0,0.0,2.000000,4.300000,21522.000000
75%,35.500000,23.400000,25.800000,21.000000,36.500000,21.500000,91.050000,0.0,0.0,2.700000,5.600000,34888.000000
max,103.100000,27.100000,30.200000,26.500000,87.000000,63.000000,97.000000,0.0,0.0,4.600000,8.300000,92609.000000


In [11]:
# sunshine_rate가 0인 모든 sample들은 sunshine_sum의 값이 NaN 또는 0이다. (train)
train[train['sunshine_rate'] == 0]['sunshine_sum'].unique()

array([nan,  0.])

In [12]:
# sunshine_rate가 0인 모든 sample들은 sunshine_sum의 값이 NaN 또는 0이다. (test) : train과 test의 sunshine_sum 결측값을 0으로 대치하자.
test[test['sunshine_rate'] == 0]['sunshine_sum'].unique()

array([nan,  0.])

In [13]:
# sunshine_sum 결측값을 0으로 대치 (train)
train['sunshine_sum'] = train[['sunshine_sum']].fillna(0)

In [14]:
# sunshine_sum 결측값을 0으로 대치 (test)
test['sunshine_sum'] = test[['sunshine_sum']].fillna(0)

### precipitation

- "자료가 공백인 곳은 관측하지 않은 지점들입니다."
- 출처 : https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-element.do?stn=108&yy=2018&obs=21

In [15]:
# precipitation 결측값을 0으로 대치 (train)
train['precipitation'] = train[['precipitation']].fillna(0)

In [16]:
# precipitation 결측값을 0으로 대치 (test)
test['precipitation'] = test[['precipitation']].fillna(0)

## 결측치 처리 결과

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1026 entries, 0 to 1094
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1026 non-null   object 
 1   precipitation  1026 non-null   float64
 2   temp_mean      1026 non-null   float64
 3   temp_highest   1026 non-null   float64
 4   temp_lowest    1026 non-null   float64
 5   PM10           1026 non-null   float64
 6   PM2.5          1026 non-null   float64
 7   humidity       1026 non-null   float64
 8   sunshine_sum   1026 non-null   float64
 9   sunshine_rate  1026 non-null   float64
 10  wind_mean      1026 non-null   float64
 11  wind_max       1026 non-null   float64
 12  rental         1026 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 112.2+ KB


In [18]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           365 non-null    object 
 1   precipitation  365 non-null    float64
 2   temp_mean      365 non-null    float64
 3   temp_highest   365 non-null    float64
 4   temp_lowest    365 non-null    float64
 5   PM10           365 non-null    float64
 6   PM2.5          365 non-null    float64
 7   humidity       365 non-null    float64
 8   sunshine_sum   365 non-null    float64
 9   sunshine_rate  365 non-null    float64
 10  wind_mean      365 non-null    float64
 11  wind_max       365 non-null    float64
dtypes: float64(11), object(1)
memory usage: 34.3+ KB
